CNN version 4.0
=============
Using the preprocessed Data
------------

Building upon the previous cnn version. 

This notebook uses a more sophisticated and a deeper cnn than the previous.
Use of TensorBoard for visualization

In [17]:
# These are all the modules we'll be using later. Make sure you can import them
# before proceeding further.
from __future__ import print_function
import os
import numpy as np
import tensorflow as tf
from six.moves import cPickle as pickle
from six.moves import range

# Task 1: Load the data from the pickle file

In [18]:
'''Set the constant paths here'''
# set the paths required for the script to work:
root = "../Data/"
pickle_file_path = root + "Data_pindown.pickle"
log_path = root + ".logs"

Retrieve the saved data from the pickle file

In [19]:
Formatted_Data = {}
with open(pickle_file_path, "rb") as f:
    Formatted_Data = pickle.load(f)

In [20]:
len(Formatted_Data)

7

In [21]:
# recollect the fragmented data

labels_mappings = Formatted_Data['label_mapping']
print(labels_mappings)

train_dataSet = Formatted_Data['train_data']
train_labels  = Formatted_Data['train_labels']

cv_dataSet = Formatted_Data['cv_data']
cv_labels = Formatted_Data['cv_labels']

test_dataSet = Formatted_Data['test_data']
test_labels  = Formatted_Data['test_labels']

print(train_dataSet.shape, cv_dataSet.shape, test_dataSet.shape)
print(train_labels.shape, cv_labels.shape, test_labels.shape)

{'Action-Painting': 0, 'Colour-Field-Painting': 1, 'Surreal-Surrealism': 8, 'art--Expressionism': 9, 'Op-Art': 6, 'Hard-Edge-Art': 3, 'ORPHISM': 5, 'Post-impressionism': 7, 'Minimalism-Art': 4, 'Cubism': 2}
(1927, 100, 100, 3) (276, 100, 100, 3) (551, 100, 100, 3)
(1927, 10) (276, 10) (551, 10)


# Task 2: Spawn the CNN 

In [22]:
# The accuracy function to calculate the accuracy of the classifier

def accuracy(predictions, labels):
    return (100.0 * np.sum(np.argmax(predictions, 1) == np.argmax(labels, 1)) / predictions.shape[0])

In [23]:
# define the constants required hence forth:
batch_size = 500
patch_size = 5
num_hidden = 256

image_size = train_dataSet.shape[1]
num_labels = len(labels_mappings)
num_channels = 3 # rgb images

### The CNN tensorflow graph 

In [24]:
'''Functions required for setting up the convolution and pooling layers of the neural network'''

def weightVariable(shape, Wname="W"):
    initial = tf.truncated_normal(shape, stddev = 0.1)
    return tf.Variable(initial, name=Wname)
        
def biasVariable(shape, Bname="B"):    
    initial = tf.zeros(shape)
    return tf.Variable(initial, name=Bname)
        
def addConvLayer(inp, shape, act_fun=tf.nn.relu):
    with tf.name_scope("Conv_Layer"):
        with tf.name_scope("Parameters"):
            # create the weight variable:
            layer_weight = weightVariable(shape, Wname="Filters")

            # create bias variable:
            layer_bias = biasVariable([shape[-1]], Bname="Filter_biases")

        return act_fun(tf.nn.conv2d(inp, layer_weight, [1, 1, 1, 1], padding="SAME") + layer_bias)
    
def addPoolLayer(inp, kernel, stride):
    return tf.nn.max_pool(inp, ksize=kernel, strides=stride, padding='SAME')
    
def addFCLayer(inp, shape, act_fun=tf.nn.relu):
    with tf.name_scope("Fully_Connected_Layer"):
        layer_weight = weightVariable(shape)
        layer_bias = biasVariable([shape[-1]])

        if(act_fun == None): 
            return tf.matmul(inp, layer_weight) + layer_bias
        # else:
        return act_fun(tf.matmul(inp, layer_weight) + layer_bias)

In [37]:
graph = tf.Graph()

''' Neural network name: BotNet
    Size: medium
    conf: conv => conv => pool => conv => pool => fc => fc => softmax'''

with graph.as_default():
    
    # Input data.
    with tf.name_scope("INPUTS"):
        tf_train_dataset = tf.placeholder(tf.float32, shape=(batch_size, image_size, image_size, num_channels), name="Data")
        tf_train_labels = tf.placeholder(tf.float32, shape=(batch_size, num_labels), name="Labels")

        tf_valid_dataset = tf.constant(cv_dataSet, name="cross_validation_dataset")
        tf_test_dataset = tf.constant(test_dataSet, name="test_dataset")
  
    # Model.
    def model(data):
        # CONV LAYER
        # layer has 8 (3 x 3) convoluting filters
        layer1_1 = addConvLayer(data, [patch_size, patch_size, num_channels, 8], tf.nn.sigmoid)
    
        
        # CONV LAYER  
        # layer has 12 (3 x 3) convoluting filters
        layer2_1 = addConvLayer(layer1_1, [patch_size, patch_size, 8, 12], tf.nn.sigmoid)
    
    
        # CONV LAYER
        # layer has 16 (3 x 3) convoluting filters
        layer3_1 = addConvLayer(layer2_1, [patch_size, patch_size, 12, 16], tf.nn.sigmoid)
    
        
        # POOL LAYER
        # will not require any data variable or parameter holder
        # a simple 2 x 2 max_pooling operation
        layer4_1 = addPoolLayer(layer3_1, [1, 2, 2, 1], [1, 2, 2, 1])
        
        # CONV LAYER
        # layer has 20 (3 x 3) convoluting filters
        layer5_1 = addConvLayer(layer4_1, [patch_size, patch_size, 16, 20], tf.nn.sigmoid)
        
        # FC LAYER
        # input dimension will be: 8 * 8 * 32
        # length, width calculation: 32 =first pool=> 16 =second pool=> 8
        # depth calculation: 32 ...(same as the last conv output)
        # output dimension: 128 ...(equal to number of hidden cells)
        # number of hidden cells: 128
        # reshape the tensor before passing it to the FC layers
        shape = layer5_1.get_shape().as_list()
        reshape = tf.reshape(layer5_1, [shape[0], shape[1] * shape[2] * shape[3]])
        layer6_1 = addFCLayer(reshape, [shape[1] * shape[2] * shape[3], num_hidden], tf.nn.sigmoid)
        
        # FC LAYER
        # input dimension will be: 256
        # output dimension will be: 256
        # number of hidden cells: 256
        layer7_1 = addFCLayer(layer6_1, [num_hidden, num_hidden], tf.nn.sigmoid)
        
        # Fianl output layer for the neural network
        layer8_1 = addFCLayer(layer7_1, [num_hidden, num_labels], tf.nn.sigmoid)
        
        return layer8_1
  
    # Training computation.
    logits = model(tf_train_dataset)
    
    with tf.name_scope("Loss"):
        loss = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits, tf_train_labels))
        tf.scalar_summary("loss", loss)
        
    # Optimizer.
    with tf.name_scope("train_step"):
        optimizer = tf.train.GradientDescentOptimizer(0.1).minimize(loss)
  
    # Predictions for the training, validation, and test data.
    with tf.name_scope("train_prediction"):
        train_prediction = tf.nn.softmax(logits)
    
    with tf.name_scope("validation_prediction"):
        validation_prediction = tf.nn.softmax(model(tf_valid_dataset))

### Graph Excuter Session module

In [38]:
# use less number of  images initially
#train_dataSet = train_dataSet[:1000]
#train_labels = train_labels[:1000]

In [39]:
num_steps = 1001
                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                
with tf.Session(graph=graph) as session:
    train_writer = tf.train.SummaryWriter(os.path.join(log_path, "train"), session.graph)
    merged = tf.merge_all_summaries()
    tf.global_variables_initializer().run()
    print('Initialized')
  
    for step in range(num_steps):
        offset = (step * batch_size) % (train_labels.shape[0] - batch_size)
        batch_data = train_dataSet[offset:(offset + batch_size), :, :, :]
        batch_labels = train_labels[offset:(offset + batch_size), :]
        
        feed_dict = {tf_train_dataset : batch_data, tf_train_labels : batch_labels}
        _, l, predictions = session.run([optimizer, loss, train_prediction], feed_dict=feed_dict)
    
        if (step % 50 == 0):
            summary = session.run(merged, feed_dict=feed_dict)
            train_writer.add_summary(summary, step)
            print('Minibatch loss at step %d: %f' % (step, l))
            print('Minibatch accuracy: %.1f%%' % accuracy(predictions, batch_labels))
            
    print('Cross Validation_Set Accuracy: %1f%%' % accuracy(validation_prediction.eval(), cv_labels))

Initialized
Minibatch loss at step 0: 2.305766
Minibatch accuracy: 8.2%
Minibatch loss at step 50: 2.284896
Minibatch accuracy: 10.2%
Minibatch loss at step 100: 2.294289
Minibatch accuracy: 10.6%
Minibatch loss at step 150: 2.283851
Minibatch accuracy: 10.0%
Minibatch loss at step 200: 2.291031
Minibatch accuracy: 11.6%
Minibatch loss at step 250: 2.284740
Minibatch accuracy: 10.6%
Minibatch loss at step 300: 2.293067
Minibatch accuracy: 10.2%
Minibatch loss at step 350: 2.286912
Minibatch accuracy: 11.8%
Minibatch loss at step 400: 2.287765
Minibatch accuracy: 10.4%
Minibatch loss at step 450: 2.287848
Minibatch accuracy: 13.2%
Minibatch loss at step 500: 2.283175
Minibatch accuracy: 11.2%
Minibatch loss at step 550: 2.286896
Minibatch accuracy: 11.4%
Minibatch loss at step 600: 2.280981
Minibatch accuracy: 11.2%
Minibatch loss at step 650: 2.293067
Minibatch accuracy: 13.2%
Minibatch loss at step 700: 2.280449
Minibatch accuracy: 11.6%
Minibatch loss at step 750: 2.293444
Minibatch 